# Code

In [1]:
print("start")

start


In [2]:
from rdkit import Chem
from rdkit.Chem import AllChem
from iotbx import cif
import MDAnalysis as mda
from pathlib import Path
import os
import numpy as np
import networkx as nx
from scipy.spatial.distance import pdist, cdist
from rdkit import Chem
from rdkit.Chem import AllChem, rdFMCS, rdDepictor, rdMolTransforms
from typing import List
from contextlib import redirect_stdout, redirect_stderr
import contextlib
from contextlib import redirect_stderr, closing
from pathlib import Path
from multiprocessing import Pool
import tqdm
from collections import Counter
import io
from io import StringIO
import warnings
import copy
import tempfile
import random
import shutil
# from elbow.utilities.options_parser import run as options_parser
# from elbow.chemistry.any_chemical_format_reader \
#      import any_chemical_format_reader
# from elbow.command_line.builder import builder
# from elbow.molfiles.molfile_parser import run as molfile_parser_run
# from elbow.chemistry.MoleculeClass import MoleculeClass
# from elbow.chemistry.any_chemical_format_reader \
#      import any_chemical_format_reader
# from elbow.command_line.builder import builder, parser, read_file
import requests
from bs4 import BeautifulSoup



class CODFile:
  @staticmethod
  def subtract_paths(path1,path2):
      p1 = list(path1.parts)
      p2 = list(path2.parts)
      for p in p1:
        p2.remove(p)
      return Path(*tuple(p2))
    
  # failure messages in the order they will be checked
  failure_messages = {
  -1:"Unknown failure",
  0:"Error parsing smiles from html file",
  1:"File does not exist",
  2:"IOTBX cif file parsing error",
  3:"Is powder diffraction",
  4:"Has no diffraction reflections",
  5:"Has no resolution information",
  6:"Resolution falls below threshold (>0.84)",
  7:"Has no R-factor information",
  8:"High or undetermined R-factor",
  9:"Failure building CCTBX xray-structure",
  10:"Contains disallwoed elements",
  11:"Number of atoms falls below cutoff (2)",
  11.5:"Number of atoms falls above cutoff (100)",
  12:"Atoms pairs clash within threshold (0.5 angstrom)",
  13:"No explicit bond information",
  14:"Mismatched atom/bond labels",
  15:"Occupancy values exist below threshold (1)",
  16:"Failure linking hydrogen atoms",
  16.5:"H bonds with valence >1",
  17:"Bond accross symmetry",
    
  18:"Failure building molecule as a networkx graph",
  19:"Failure building molecule as an rdkit molecule object",
  20:"Warning when building molecule as rdkit",
  21:"Failure building molecule as elbow molecule",
  22:"Failed converting from elbow to rdkit",
  23:"Failed assigning bond orders"}
#   21:"Failed making generic rdkit molecule",
#   22:"Failed making eLBOW molecule",
#   23:"Failed splitting into disconnected mols"}
  
  def __init__(self,filepath,cod_id=None,debug=False):
    filepath = Path(filepath)
    if cod_id == None:
      cod_id = filepath.name
    self.cod_id = cod_id
    self.filepath = filepath
    self.warnings = [] # a list of error/warning strings
    pt  = Chem.GetPeriodicTable()
    self.total_elements = set([pt.GetElementSymbol(i+1) for i in range(118)])
    self.allowed_elements = set([pt.GetElementSymbol(i+1) for i in range(35)])
    failed = False
    fail_message = None
    

    ########### Check if file exists
    
    if not self.filepath.exists():
      failed = True
      fail_message = self.failure_messages[1]
      
      
    ########### Parse file
    if not failed:
      try:
          cif_model = cif.reader(self.filepath.as_posix()).model()
          code = cif_model.keys()[0]
          self.cif_model = cif_model[code]

      except:
        self.cif_model = None
        failed = True
        fail_message = self.failure_messages[2]
        
    cif_key_set= set(self.cif_model.keys())
    
    
    ########### Check Powder diffraction
    if not failed:
      
      if any(["_pd_" in key for key in cif_key_set]):
        failed = True
        fail_message = self.failure_messages[3]
    
    
    ########### Check Powder diffraction
    
    if not failed:
      if not any(["_diffrn_reflns_" in key for key in cif_key_set]):
        failed=True
        fail_message = self.failure_messages[4]
    
    ########### Check for Resolution info
    
    if not failed:
      resolution_keys = {"_diffrn_radiation_wavelength","_diffrn_reflns_theta_max"}
      if len(cif_key_set.intersection(resolution_keys)) !=2:
        failed = True
        fail_message = self.failure_messages[5]
    
    ########### Check Resolution info
    if not failed:
      
      try:
        wavelength_str = self.cif_model["_diffrn_radiation_wavelength"]
        theta_max_str = self.cif_model["_diffrn_reflns_theta_max"]
        wavelength = float(np.array(wavelength_str))
        theta_max = float(np.array(theta_max_str))
        sin_theta = np.sin(theta_max)
        if np.isclose(sin_theta,0) or not np.isfinite(sin_theta) or not np.isfinite(wavelength):
          failed = True
          fail_message =self.failure_messages[5]
        if not failed:
          dmax = wavelength/(2*np.sin(theta_max))
          if dmax > 0.84:
            failed = True
            fail_message = self.failure_messages[6]
      except:
        failed = True
        fail_message = self.failure_messages[5]

        
    ########### Check R factor info
    if not failed:
      Rfactor_cif_keys = set(["_refine_ls_R_factor_all","_refine_ls_R_factor_gt"])

      if cif_key_set.intersection(Rfactor_cif_keys)==0:
        failed = True
        fail_message = self.failure_messages[7]
    
    
    ########### Check R factor values
    if not failed:   
      R_values = []
      if "_refine_ls_R_factor_gt" in self.cif_model.keys():
        R_values.append(self.cif_model["_refine_ls_R_factor_gt"])
      if "_refine_ls_R_factor_all" in self.cif_model.keys():
        R_values.append(self.cif_model["_refine_ls_R_factor_all"])
      try:
        R_values = np.array(R_values,dtype=float)

        if np.any(R_values>0.1):
          failed=True
          fail_message = self.failure_messages[8]

      except:
        failed=True
        fail_message = self.failure_messages[8]

    
    ########### Try building xray structure
    if not failed:
      try:
        self.xs = cif.builders.crystal_structure_builder(self.cif_model).structure
      except:
        self.xs = None
        failed = True
        fail_message = self.failure_messages[9]

    
    ########### Reject elements
    if not failed:
      try:
        self.element_set = set(["H" if sc.element_symbol()=="D" else sc.element_symbol() for sc in self.xs.scatterers()])
        if not self.element_set.issubset(self.allowed_elements):
          failed = True
          fail_message = self.failure_messages[10]

      except:
        failed = True
        fail_message = self.failure_messages[10]
    
    
    ########### Get atomic numbers
    if not failed:
      try:
        pt  = Chem.GetPeriodicTable()
        self.atomic_names = ["H" if sc.element_symbol()=="D" else sc.element_symbol() for sc in self.xs.scatterers()]
        self.atomic_numbers = [pt.GetAtomicNumber(element) for element in self.atomic_names]
        self.xyz = self.xs.sites_cart().as_numpy_array()

      except:
        failed = True
        fail_message = self.failure_messages[10]

        
    ########### Check minimum size
    if not failed:
      if len(self.atomic_numbers)<=2:
        failed = True
        fail_message = self.failure_messages[11]
    
    ########### Check maximum size
    if not failed:
      if len(self.atomic_numbers)>=100:
        failed = True
        fail_message = self.failure_messages[11.5]
    
    
    ########### Check minimum atom pair distance
    if not failed:
      dists = pdist(self.xyz)
      if len(dists) <1 or not np.all(np.isfinite(dists)):
        failed = True
        fail_message = self.failure_messages[12]
      if not failed:
        if np.min(dists)<=0.5:
          failed = True
          fail_message = self.failure_messages[12]

    ########### Check for explicit bonds
    if not failed:
      if "_geom_bond_atom_site_label_1" not in self.cif_model.keys() or "_geom_bond_atom_site_label_2" not in self.cif_model.keys():
        failed = True
        fail_message = self.failure_messages[13]

    ########### Check mismatched atom/bond labels
    if not failed:
      cif_model = self.cif_model
      site_labels = list(cif_model["_atom_site_label"])
      labels = {label for pair in zip(cif_model["_geom_bond_atom_site_label_1"],cif_model["_geom_bond_atom_site_label_2"]) for label in pair}
      if not labels.issubset(set(site_labels)):
        failed = True
        fail_message = self.failure_messages[14]

    ########### Check low occupancy
    if not failed:
      occupancy = np.array([sc.occupancy for sc in self.xs.scatterers()])
      if np.any(occupancy<1.0):
        failed = True
        fail_message = self.failure_messages[15]
       
    

    ########### check for bonds accross symmetry
    if not failed:
      key = '_geom_bond_site_symmetry_2'
      try:
        bond_sym = list(self.cif_model[key])
        if len(list(set(bond_sym)))>1:
          failed = True
          fail_message = self.failure_messages[17]
      except:
        pass
    
        
    ########### Build nx graph
    if not failed:
      try:
        cif_model = self.cif_model
        site_labels = list(cif_model["_atom_site_label"])
        label_dict_label_keys = {label:i for i,label in enumerate(site_labels)}
        label_dict_idx_keys = {i:label for i,label in enumerate(site_labels)}
        xs = self.xs
        G = nx.Graph()
        for i,(sc,xyz) in enumerate(zip(xs.scatterers(),xs.sites_cart())):
          G.add_node(i)

        for label1,label2 in zip(cif_model["_geom_bond_atom_site_label_1"],cif_model["_geom_bond_atom_site_label_2"]):
          idx1,idx2 = label_dict_label_keys[label1], label_dict_label_keys[label2]
          G.add_edge(idx1,idx2)

        self.atom_graph = G
      except:
        failed = True
        fail_message = self.failure_messages[18]  
           
    ########### check if H bonds are explicit
    if not failed:
      try:
        non_explicit_Hbonds = []
        excessive_valence_hbonds = []
        for atom in self.atom_graph:
          atomic_number = self.atomic_numbers[atom]
          if atomic_number==1:
            if len(self.atom_graph.edges(atom)) == 0:
              non_explicit_Hbonds.append(atom)
            elif len(self.atom_graph.edges(atom)) >1:
              excessive_valence_hbonds.append(atom)
        
        if len(excessive_valence_hbonds)>0:
          failed = True
          fail_message = self.failure_messages[16.5]
        
        if not failed:
          if len(non_explicit_Hbonds)>0:
            dist = cdist(self.xyz,self.xyz)
            for atomi in non_explicit_Hbonds:
              partner = np.argwhere(dist[atomi]==np.sort(dist[atomi])[1])[0][0]
              self.atom_graph.add_edge(atomi,partner)
      except:
        failed = True
        fail_message = self.failure_messages[16]     



        

    ########### Build RDKIT Molecules      
    
    if not failed:
      try:
        self.build_rdkit()
        self.build_rdkit_single()
      except:
        failed = True
        fail_message = self.failure_messages[19]
        
#     ########### Assign bond order   
#     if not failed:
#       #try:
#       _, success = self.assign_bond_orders(self)
#       if not success:
#         failed = True
#         fail_message = self.failure_messages[23]
#       # except:
#       #   failed = True
#       #   fail_message = self.failure_messages[23]
      

#     if not failed:
#       try:
#         Chem.WrapLogs()
#         AllChem.WrapLogs()
#         with redirect_stderr(io.StringIO()) as err:
#           failed = False
#           smiles_mol = self.smiles
#           smiles_mol = Chem.AddHs(smiles_mol)
#           newmol = None
#           success = False
#           try:
#             newmol, perfect_match = PercieveBondOrderFromTemplate(smiles_mol,self.mol_single)
#             if perfect_match:
#               success = True
#           except:
#             pass
  
    
#     ########## Make Elbow molecules
#     if not failed:
#       try:
#         with redirect_stdout(io.StringIO()) as out:
#           with redirect_stderr(io.StringIO()) as err:
#             self.elbow_mol_strings = []
#             #self.elbow_pdb_strings = []
#             for oldmol in self.mols:
#               mblock = Chem.MolToMolBlock(oldmol)
#               molecule = any_chemical_format_reader(mblock,simple=False)
#               assert len(molecule) == oldmol.GetNumAtoms()
#               molecule.Bondise(add_bond_order=True)
#               molecule.PropogateBondOrders()
#               molecule.Chargise()
#               molecule.Peptidise()
#               molecule.Acidise()
#               molecule.Ringise()
#               sdf = molecule.WriteSDF2String()
#               sdf_lines = sdf.split("\n")
#               end_lines = [i for i,line in enumerate(sdf_lines) if "M END" in line]
#               assert len(end_lines)==1
#               end_line = end_lines[0]
#               sdf = "\n".join(sdf_lines[:end_line]+["M  END"])
#               new_mblock = sdf
              
#               self.elbow_mol_strings.append(sdf)
#               #self.elbow_pdb_strings.append(molecule.WritePDBLigand2String(hydrogens=True))
#               self.warnings.append(err.getvalue())
  #         Chem.WrapLogs()
  #         with redirect_stderr(io.StringIO()) as err:
  #           with redirect_stdout(io.StringIO()) as out:
  #             Chem.WrapLogs()
  #             mol_from_mol2 = Chem.MolFromMol2Block(mol2_string,removeHs=False)
  #             newmol = AllChem.AssignBondOrdersFromTemplate(mol_from_mol2,oldmol)
  #             assert np.all(np.isclose(newmol.GetConformer().GetPositions(),oldmol.GetConformer().GetPositions()))
  #             new_mols.append(newmol)
          
#       except:
#         failed = True
#         fail_message = self.failure_messages[21]

    ######## Convert from elbow strings to rdkit
#     if not failed:
# #       try:
#         Chem.WrapLogs()
#         self.new_mols = []
        
#         for oldmol,mol_string in zip(self.mols,self.elbow_mol_strings):
#           with redirect_stderr(io.StringIO()) as err:
#             newmol = Chem.MolFromMolBlock(mol_string,removeHs=False)
            
#             assert oldmol.GetNumAtoms() == newmol.GetNumAtoms()
#             assert np.all(np.isclose(newmol.GetConformer().GetPositions(),oldmol.GetConformer().GetPositions(),atol=1e-02))
#             new_conf = newmol.GetConformer()
#             newmol.RemoveAllConformers()
#             _ = newmol.AddConformer(oldmol.GetConformer())
#             self.new_mols.append(newmol)
#       except:
#         failed = True
#         fail_message = self.failure_messages[22]
      
    ########## Cleanup
#     if not failed:
#       self.mols = self.new_mols
#       del self.elbow_mol2_strings
  
  
#       if self.write_output:
#         if not failed:
#           try:
#             self.write_output_files()
#           except:
#             self.failed = True
#             self.fail_message = self.failure_messages[22]
        
    ########### Finished
    
    self.failed = failed
    self.fail_message = fail_message
    
    if debug:
      for key,value in locals().items():
        if not hasattr(self,key):
          setattr(self,key,value)


    
  
  
  def build_rdkit(self):
    pt  = Chem.GetPeriodicTable()
    Chem.WrapLogs()
    with redirect_stderr(io.StringIO()) as err:
      G = self.atom_graph
      xs = self.xs    
      mols = []
      pdb_strings = []
      for subgraph in list(nx.connected_components(G)):
        mol = Chem.Mol()
        rwmol = Chem.RWMol(mol)
        conformer = Chem.Conformer(len(subgraph))

        new_idx_mapping = {}
        bond_pairs = set()

        for i,atom in enumerate(subgraph):
          new_idx_mapping[atom]=i
          sc = xs.scatterers()[atom]
          xyz = xs.sites_cart()[atom]
          sc = ("H" if sc.element_symbol()=="D" else sc.element_symbol())
          atom = rwmol.AddAtom(Chem.Atom(pt.GetAtomicNumber(sc)))
          conformer.SetAtomPosition(atom,xyz)

        for atom in subgraph:
          for edge in G.edges(atom):
            start,end = edge
            if start!=end:
              start, end = new_idx_mapping[start], new_idx_mapping[end]
              bond_pairs.add(tuple(sorted([start,end])))

        for bond_pair in bond_pairs:
          idx1,idx2 = bond_pair
          rwmol.AddBond(idx1,idx2,Chem.BondType.SINGLE)

        rwmol.AddConformer(conformer)  
        mol = rwmol.GetMol()

        mols.append(mol)
     
    self.mols = mols
    
    warning = err.getvalue()
    if len(warning)>0:
      self.warnings.append(warning)
      self.failed = True
      self.fail_message = self.failure_messages[20]
      
  def write_output_files(self):

    def subtract_paths(path1,path2):
      p1 = list(path1.parts)
      p2 = list(path2.parts)
      for p in p1:
        p2.remove(p)
      return Path(*tuple(p2))


    cif_file =  self.filepath
    out_cif_path = Path(out_dir,subtract_paths(cod_dir,cif_file))
    entry_out_dir = Path(out_cif_path.parent,out_cif_path.stem)
    entry_id = entry_out_dir.stem
    out_cif_path = Path(entry_out_dir,entry_id+".cif")

    if entry_out_dir.exists():
      shutil.rmtree(entry_out_dir)
    entry_out_dir.mkdir(parents=True)
    out_cif_path.symlink_to(cif_file)

#     for i,mol in enumerate(self.mols):
#       stem = entry_id+"_PRE_SUB_"+str(i).zfill(3)
#       pdb_path = Path(entry_out_dir,stem+".pdb")
#       Chem.MolToPDBFile(mol,pdb_path.as_posix())

    for i,mol in enumerate(self.new_mols):
      stem = entry_id+"_SUB_"+str(i).zfill(3)
      path = Path(entry_out_dir,stem+".mol")
      Chem.MolToMolFile(mol,path.as_posix())
      
  def build_rdkit_single(self):
    pt  = Chem.GetPeriodicTable()
    G = self.atom_graph
    atoms = list(G.nodes())
    xs = self.xs    

    
    mol = Chem.Mol()
    rwmol = Chem.RWMol(mol)
    conformer = Chem.Conformer(len(atoms))

    new_idx_mapping = {}

    bond_pairs = set()
    elements = []


    for i,atom in enumerate(atoms):
      new_idx_mapping[atom]=i
      sc = xs.scatterers()[atom]
      xyz = xs.sites_cart()[atom]
      element = ("H" if sc.element_symbol()=="D" else sc.element_symbol())
      elements.append(element)
      atomic_num = pt.GetAtomicNumber(element)
      atom = rwmol.AddAtom(Chem.Atom(atomic_num))
      conformer.SetAtomPosition(atom,xyz)

    for atom in atoms:
      for edge in G.edges(atom):
        start,end = edge
        if start!=end:
          start, end = new_idx_mapping[start], new_idx_mapping[end]
          bond_pairs.add(tuple(sorted([start,end])))

    for bond_pair in bond_pairs:
      idx1,idx2 = bond_pair
      e1,e2 = elements[idx1],elements[idx2]
      rwmol.AddBond(idx1,idx2,Chem.BondType.SINGLE)

    rwmol.AddConformer(conformer)
    mol = rwmol.GetMol()
    self.mol_single = mol

  @staticmethod
  def guess_bond_order_mda(mol):
    newmol = None
    success = False
    skip = (len(mol.GetBonds())==0)
    if skip:
      success = True
    else:
      try:
        m = copy.deepcopy(mol)
        params = Chem.rdmolops.AdjustQueryParameters()
        params.makeBondsGeneric = True
        modmol = Chem.rdmolops.AdjustQueryProperties(m, params)
        modmol.UpdatePropertyCache()
        pdb_string= Chem.MolToPDBBlock(modmol)
        tmp = tempfile.NamedTemporaryFile('w+t')
        tmp.write(pdb_string)
        tmp.seek(0)
        u= mda.Universe(tmp.name,format="PDB")
        tmp.close()
        elements = mda.topology.guessers.guess_types(u.atoms.names)
        u.add_TopologyAttr('elements', elements)
        element_set = set([a.GetSymbol() for a in mol.GetAtoms()])
        force = ("H" not in element_set)
        newmol = u.atoms.convert_to("RDKIT",force=force)
        success = True
      except:
        success = False
    return newmol,success


  @staticmethod
  def assign_bond_orders(codfile):
    
    pieces_codfile = codfile.mols
    failed = False
    fixed_pieces = []
    for p in pieces_codfile:
      new_mol, success = codfile.guess_bond_order_mda(p)
      if success and (new_mol is not None):
        fixed_pieces.append(new_mol)
      else:
        failed = True

    if not failed:
      if len(fixed_pieces)!= len(codfile.mols):
        failed = True
        print("fixed pieces doesn't match size of codfile.mols",len(fixed_pieces),len(codfile.mols))
          
        codfile.mols = fixed_pieces
        
    return codfile, not failed


#   def RDKIT_make_generic(self,rdmol):
#     Chem.WrapLogs()
#     with redirect_stderr(io.StringIO()) as err:
#       mol = copy.deepcopy(rdmol)
#       params = Chem.rdmolops.AdjustQueryParameters()
#       params.makeBondsGeneric = True
#       modmol = Chem.rdmolops.AdjustQueryProperties(mol, params)
#       modmol.UpdatePropertyCache()

#       for atom in modmol.GetAtoms():
#         atom.SetFormalCharge(0)
#       modmol.UpdatePropertyCache()
#     string_err = err.getvalue()
#     return modmol, string_err

In [3]:
import sys
sys.path.append("../")
from phenixml.visualization.py3d_mol_wrappers import showmol

# Read cod files

In [4]:
processing_dir = Path("/dev/shm/cschlick/COD_processing/")
raw_dir = Path(processing_dir,"COD_raw")
cif_files = [path for path in raw_dir.glob("**/*") if path.suffix == ".cif"]
print("Number cif files:",len(cif_files))

Number cif files: 467767


In [5]:

# with open("codfiles.txt","r") as fh:
#   codids= fh.readlines()
#   codids = [file.strip("\n") for file in codids]
  
# cleared_cod_ids = set([Path(codid).stem for codid in codids])
# useful_cif_files = [file for file in cif_files if file.stem in cleared_cod_ids]



# take all
useful_cif_files = cif_files
print("Number useful cif files:",len(useful_cif_files))

Number useful cif files: 467767


In [6]:
work = useful_cif_files
def worker(data):
  try:
    with redirect_stderr(StringIO()) as err:
      codfile = CODFile(data)
    return codfile
  except:
    return None


In [7]:
with closing(Pool(processes=32)) as pool:
  results = []
  for result in tqdm.tqdm(pool.imap_unordered(worker, work), total=len(work)):
      results.append(result)
  pool.terminate()

100%|██████████| 467767/467767 [33:18<00:00, 234.06it/s]  


In [8]:
results = [result for result in results if result is not None]

In [9]:
failed_codfiles = [codfile for codfile in results if codfile.failed]
failed_messages = [codfile.fail_message for codfile in results if codfile.failed]
kept_codfiles = [codfile for codfile in results if not codfile.failed]
print("Failed:",len(failed_codfiles))
print("Kept:",len(kept_codfiles))


fail_counter = {message:0 for message in CODFile.failure_messages.values()}
for message in failed_messages:
  fail_counter[message]+=1
  
print("\nReasons for failure:")
fail_check = 0
for key,value in fail_counter.items():
  print("  ",key.ljust(60," "),":",value)
  fail_check+=value
print("Fail check:",fail_check)

Failed: 388657
Kept: 79098

Reasons for failure:
   Unknown failure                                              : 0
   Error parsing smiles from html file                          : 0
   File does not exist                                          : 0
   IOTBX cif file parsing error                                 : 0
   Is powder diffraction                                        : 4447
   Has no diffraction reflections                               : 71561
   Has no resolution information                                : 1171
   Resolution falls below threshold (>0.84)                     : 60473
   Has no R-factor information                                  : 0
   High or undetermined R-factor                                : 69632
   Failure building CCTBX xray-structure                        : 261
   Contains disallwoed elements                                 : 99192
   Number of atoms falls below cutoff (2)                       : 56
   Number of atoms falls above cutoff (100

In [16]:
print("hi")

hi


In [ ]:
def subtract_paths(path1,path2):
    p1 = list(path1.parts)
    p2 = list(path2.parts)
    for p in p2:
      p1.remove(p)
    return Path(*tuple(p1))

In [ ]:
filtered_dir = Path(processing_dir,"COD_filtered")
if filtered_dir.exists():
  shutil.rmtree(filtered_dir)

In [ ]:
mol_datas = [{"ciffile":codfile.filepath,"mol":codfile.mol_single} for codfile in kept_codfiles]

In [ ]:
fail_reasons = {0:"none",
               1:"Explicit valence error",
               2:"Kekulize error",
               3:"unknown"}

def worker(mol_data):
  failed = False
  with redirect_stderr(StringIO()) as err:
    mol = mol_data["mol"]
    m = Chem.MolFromMolBlock(Chem.MolToMolBlock(mol),removeHs=False,sanitize=False)
    err = err.getvalue()
    mol_data["sanitize_error"] = err
    if "Explicit valence" in err:
      failed = True
      mol_data["fail_message"] = fail_reasons[1]
    elif "Can't kekulize" in err:
      failed = True
      mol_data["fail_message"] = fail_reasons[2]
    elif len(err)>0:
      failed = True
      mol_data["fail_message"] = fail_reasons[3]
    else:
      mol_data["fail_message"] = fail_reasons[0]

      outcif = Path(filtered_dir,subtract_paths(mol_data["ciffile"],raw_dir))
      outmol = outcif.with_suffix(".mol")
      outmol.parent.mkdir(exist_ok=True,parents=True)
      Chem.MolToMolFile(mol,str(outmol))
  return mol_data

In [ ]:
work = mol_datas
with closing(Pool(processes=16)) as pool:
  results = []
  for result in tqdm.tqdm(pool.imap_unordered(worker, work), total=len(work)):
      results.append(result)
  pool.terminate()

100%|██████████| 79098/79098 [00:52<00:00, 1495.66it/s]


In [ ]:
sanitize_success = [mdata for mdata in results if mdata["fail_message"]=="none"]
failed_messages = [mdata["fail_message"] for mdata in results]
fail_counter = {message:0 for message in fail_reasons.values()}
for message in failed_messages:
  fail_counter[message]+=1
  
print("\nReasons for failure:")
fail_check = 0
for key,value in fail_counter.items():
  print("  ",key.ljust(60," "),":",value)
  fail_check+=value
print("Fail check:",fail_check)


Reasons for failure:
   none                                                         : 76684
   Explicit valence error                                       : 0
   Kekulize error                                               : 0
   unknown                                                      : 2414
Fail check: 79098


In [104]:
work = [mdata["incif"] for mdata in final_success]

In [105]:
import requests
from bs4 import BeautifulSoup
import time


def worker(cif_file):
  file_name = cif_file.stem+".html"
  file_path = Path(cif_file.parent,file_name)
  if not file_path.exists():
    r = requests.get('http://www.crystallography.net/cod/'+file_name)
    if r.status_code == 200:
      with file_path.open("w") as fh:
        fh.write(r.text)
        
with closing(Pool(processes=8)) as pool:
  results = []
  for result in tqdm.tqdm(pool.imap_unordered(worker, work), total=len(work)):
      results.append(result)
  pool.terminate()

100%|██████████| 61312/61312 [13:02<00:00, 78.32it/s]  


In [28]:
html_files = [path for path in raw_dir.glob("**/*") if path.suffix == ".html"]
print("Number html files:",len(html_files))

Number html files: 0


In [114]:
def get_smiles(request_text):
  try:
    soup = BeautifulSoup(request_text, 'html.parser')
    tags = soup.find_all("th",string="SMILES")
    tag = tags[0]
    s = tag.find_next_siblings()[0]
    smiles = str(s.contents[0])
    return smiles
  except:
    return None

    

def worker(html_file):
  with html_file.open("r") as fh:
    text = fh.read()
    cod_id = html_file.stem
    smiles = get_smiles(text)
    if smiles is not None:
      
      smifile = html_file.with_suffix(".smi")
      with smifile.open("w") as fh:
        fh.write(smiles)

  return (cod_id,smiles)

In [116]:
work = html_files
with closing(Pool(processes=16)) as pool:
  results = []
  for result in tqdm.tqdm(pool.imap_unordered(worker, work), total=len(work)):
      results.append(result)
  pool.terminate()

100%|██████████| 76593/76593 [02:39<00:00, 480.89it/s]  


In [6]:
filtered_dir = Path("/dev/shm/cschlick/COD_processing/COD_filtered")
filtered_mols = [path for path in filtered_dir.glob("**/*") if path.suffix == ".mol"]
smiles = [path for path in raw_dir.glob("**/*") if path.suffix == ".smi"]

print("Number filtered mol files:",len(filtered_mols))
print("Number of smiles files:",len(smiles))

Number filtered mol files: 76684
Number of smiles files: 65011


In [8]:
mdatas = {}
for mol in filtered_mols:
  cod_id = mol.stem
  mdatas[cod_id] = {"molfile":mol,"cod_id":cod_id}
  
for smi in smiles:
  cod_id = smi.stem
  if cod_id in mdatas:
    mdatas[cod_id]["smifile"]=smi
    
filtered_mdatas = {key:value for key,value in mdatas.items() if "smifile" in value and "molfile" in value}
print("Number mol files with smiles:",len(filtered_mdatas))

Number mol files with smiles: 58931


In [9]:
fail_reasons = {0:"none",
               1:"Explicit valence error",
               2:"Kekulize error",
               3:"unknown"}


for i,(cod_id,mdata) in enumerate(filtered_mdatas.items()):
  failed = False
  with redirect_stderr(StringIO()) as err:
   
    with Path(mdata["smifile"]).open('r') as fh:
      smiles = fh.read()
    ref_mol = Chem.MolFromSmiles(smiles,sanitize=True)
    
    err = err.getvalue()
    mdata["sanitize_error"] = err
    if "Explicit valence" in err:
      failed = True
      mdata["fail_message"] = fail_reasons[1]
    elif "Can't kekulize" in err:
      failed = True
      mdata["fail_message"] = fail_reasons[2]
    elif len(err)>0:
      failed = True
      mdata["fail_message"] = fail_reasons[3]
    else:
      mdata["fail_message"] = fail_reasons[0]
  

  

In [10]:
success = [mdata for cod_id,mdata in filtered_mdatas.items() if mdata["fail_message"]=="none"]
failed_messages = [mdata["fail_message"] for cod_id,mdata in filtered_mdatas.items()]
fail_counter = {message:0 for message in fail_reasons.values()}
for message in failed_messages:
  fail_counter[message]+=1
  
print("\nReasons for failure:")

for key,value in fail_counter.items():
  print("  ",key.ljust(60," "),":",value)


Reasons for failure:
   none                                                         : 54102
   Explicit valence error                                       : 4243
   Kekulize error                                               : 586
   unknown                                                      : 0


In [11]:
# using smiles
from rdkit.Chem.AllChem import *
import warnings
import time
  
def has_match(refmol,mol):
  refmol2 = rdchem.Mol(refmol)
  mol2 = rdchem.Mol(mol)
  # do the molecules match already?
  matching = mol2.GetSubstructMatch(refmol2)
  if not matching:  # no, they don't match
    # check if bonds of mol are SINGLE
    for b in mol2.GetBonds():
      if b.GetBondType() != BondType.SINGLE:
        b.SetBondType(BondType.SINGLE)
        b.SetIsAromatic(False)
    # set the bonds of mol to SINGLE
    for b in refmol2.GetBonds():
      b.SetBondType(BondType.SINGLE)
      b.SetIsAromatic(False)
    # set atom charges to zero;
    for a in refmol2.GetAtoms():
      a.SetFormalCharge(0)
      a.SetNumRadicalElectrons(0)
    for a in mol2.GetAtoms():
      a.SetFormalCharge(0)
      a.SetNumRadicalElectrons(0)

    matching = mol2.GetSubstructMatches(refmol2, uniquify=False)
  # do the molecules match now?
  if matching:
    return True
  else:
    return False

def assign_bond_orders_from_reference(refmol,mol,sanitize=True):
  refmol2 = rdchem.Mol(refmol)
  mol2 = rdchem.Mol(mol)
  # do the molecules match already?
  matching = mol2.GetSubstructMatch(refmol2)
  if not matching:  # no, they don't match
    # check if bonds of mol are SINGLE
    for b in mol2.GetBonds():
      if b.GetBondType() != BondType.SINGLE:
        b.SetBondType(BondType.SINGLE)
        b.SetIsAromatic(False)
    # set the bonds of mol to SINGLE
    for b in refmol2.GetBonds():
      b.SetBondType(BondType.SINGLE)
      b.SetIsAromatic(False)
    # set atom charges to zero;
    for a in refmol2.GetAtoms():
      a.SetFormalCharge(0)
      a.SetNumRadicalElectrons(0)
    for a in mol2.GetAtoms():
      a.SetFormalCharge(0)
      a.SetNumRadicalElectrons(0)

    matching = mol2.GetSubstructMatches(refmol2, uniquify=False)
  # do the molecules match now?
  if matching:
    if len(matching) > 1:
      logger.warning("More than one matching pattern found - picking one")
    matching = matching[0]
    # apply matching: set bond properties
    for b in refmol.GetBonds():
      atom1 = matching[b.GetBeginAtomIdx()]
      atom2 = matching[b.GetEndAtomIdx()]
      b2 = mol2.GetBondBetweenAtoms(atom1, atom2)
      b2.SetBondType(b.GetBondType())
      b2.SetIsAromatic(b.GetIsAromatic())
    # apply matching: set atom properties
    for a in refmol.GetAtoms():
      a2 = mol2.GetAtomWithIdx(matching[a.GetIdx()])
      a2.SetHybridization(a.GetHybridization())
      a2.SetIsAromatic(a.GetIsAromatic())
      a2.SetNumExplicitHs(a.GetNumExplicitHs())
      a2.SetFormalCharge(a.GetFormalCharge())
      a2.SetNumRadicalElectrons(a.GetNumRadicalElectrons())
    if sanitize:
      SanitizeMol(mol2)
    if hasattr(mol2, '__sssAtoms'):
      mol2.__sssAtoms = None  # we don't want all bonds highlighted
  else:
    raise ValueError("No matching found")
  return mol2

fail_reasons = {0:"none",
               1:"Water without Hydrogens",
               2:"Failed assigning reference/target pieces",
               3:"Error assigning bond orders",
               4: "Number of pieces >10",
               5: "ID in blacklist"}

blacklist = ["451015","1520624","7036986","4510150"]

def worker(mdata):
  with warnings.catch_warnings(record=True) as w:
    warnings.simplefilter("always")
    with redirect_stderr(StringIO()) as err:
      with redirect_stdout(StringIO()) as out:
        cod_id = mdata["cod_id"]
        fail_message = fail_reasons[0]
        failed = False
        t0= time.time()
        if cod_id in blacklist:
          failed = True
          fail_message = fail_reasons[5]
        if not failed:
          Chem.WrapLogs()
          AllChem.WrapLogs()
          
          with Path(mdata["smifile"]).open('r') as fh:
            smiles = fh.read()
          ref_mol = Chem.MolFromSmiles(smiles,sanitize=True)
          ref_mol = Chem.AddHs(ref_mol)
          #pieces_ref = Chem.GetMolFrags(ref_mol,asMols=True,sanitizeFrags=False)
          target_mol = Chem.MolFromMolFile(str(mdata["molfile"]),removeHs=False,sanitize=False)
          if ref_mol.GetNumAtoms()==target_mol.GetNumAtoms():
            try:
              new_target = assign_bond_orders_from_reference(ref_mol,target_mol)
            except:
              failed = True
              fail_message = fail_reasons[3]
          else:
            pieces_ref = Chem.GetMolFrags(ref_mol,asMols=True,sanitizeFrags=False)
            pieces_target = Chem.GetMolFrags(target_mol,asMols=True,sanitizeFrags=False)
            piece_index_matching = {}
            if len(pieces_target)>10:
              failed = True
              fail_message = fail_reasons[4]
          
            if not failed:
              for i,ptar in enumerate(pieces_target):
                for j,pref in enumerate(pieces_ref):
                  # deal with no-hydrogen waters
                  element_set_ref = set([atom.GetAtomicNum() for atom in pref.GetAtoms()])
                  element_set_tar = set([atom.GetAtomicNum() for atom in ptar.GetAtoms()])
                  if element_set_ref == set([1,8]) and len(element_set_ref)!=len(element_set_tar):
                    failed = True
                    fail_message = fail_reasons[1]
                  if has_match(pref,ptar):
                    piece_index_matching[i]=j
            if not failed:
              if len(piece_index_matching)!=len(pieces_target):
                failed = True
                fail_message = fail_reasons[2]
              if not failed:
                new_targets = []
                for i,ptar in enumerate(pieces_target):
                  pref = pieces_ref[piece_index_matching[i]]
                  try:
                    new_target = assign_bond_orders_from_reference(pref,ptar)
                    new_targets.append(new_target)
                  except:
                    failed = True
                    fail_message = fail_reasons[3]
                if not failed:
                  new_target = new_targets[0]
                  if len(new_targets)>1:
                    for t in new_targets[1:]:
                      new_target = Chem.CombineMols(new_target,t)

               


        mdata["err"] = err.getvalue()
        mdata["out"] = out.getvalue()
        mdata["warnings"] = [str(warn.message) for warn in w]
        mdata["failed"] = failed
        mdata["fail_message"] = fail_message
        mdata["time_elapsed"] = time.time()-t0
        if not failed:
          mdata["new_target"] = new_target
        return mdata


In [12]:
work = success
with closing(Pool(processes=16)) as pool:
  results = []
  for result in tqdm.tqdm(pool.imap_unordered(worker, work), total=len(work)):
      results.append(result)
  pool.terminate()

100%|██████████| 54102/54102 [01:05<00:00, 830.83it/s] 


In [13]:
failed_messages = [mdata["fail_message"] for mdata in results]
fail_counter = {message:0 for message in fail_reasons.values()}
for message in failed_messages:
  fail_counter[message]+=1
  
print("\nReasons for failure:")
for key,value in fail_counter.items():
  print("  ",key.ljust(60," "),":",value)



Reasons for failure:
   none                                                         : 51591
   Water without Hydrogens                                      : 634
   Failed assigning reference/target pieces                     : 602
   Error assigning bond orders                                  : 1253
   Number of pieces >10                                         : 19
   ID in blacklist                                              : 3


In [14]:
bond_order_success = [mdata for mdata in results if mdata["fail_message"]=="none"]

In [15]:
fail_reasons = {0:"none",
               1:"Explicit valence error",
               2:"Kekulize error",
               3:"unknown"}


for i,mdata in enumerate(bond_order_success):
  failed = False
  with redirect_stderr(StringIO()) as err:
     
    Chem.SanitizeMol(mdata["new_target"])
    
    err = err.getvalue()
    mdata["sanitize_error"] = err
    if "Explicit valence" in err:
      failed = True
      mdata["fail_message"] = fail_reasons[1]
    elif "Can't kekulize" in err:
      failed = True
      mdata["fail_message"] = fail_reasons[2]
    elif len(err)>0:
      failed = True
      mdata["fail_message"] = fail_reasons[3]
    else:
      mdata["fail_message"] = fail_reasons[0]
  

In [16]:
failed_messages = [mdata["fail_message"] for mdata in bond_order_success]
fail_counter = {message:0 for message in fail_reasons.values()}
for message in failed_messages:
  fail_counter[message]+=1
  
print("\nReasons for failure:")
for key,value in fail_counter.items():
  print("  ",key.ljust(60," "),":",value)



Reasons for failure:
   none                                                         : 51591
   Explicit valence error                                       : 0
   Kekulize error                                               : 0
   unknown                                                      : 0


# Guess assignment

In [31]:
filtered_dir = Path("/dev/shm/cschlick/COD_processing/COD_filtered")
filtered_mols = [path for path in filtered_dir.glob("**/*") if path.suffix == ".mol"]

print("Number filtered mol files:",len(filtered_mols))

Number filtered mol files: 76684


In [32]:
mol_dicts = [{"filename":str(mol_file)} for mol_file in filtered_mols] # initialize mol dict

In [33]:
pt  = Chem.GetPeriodicTable()
total_elements = set([pt.GetElementSymbol(i+1) for i in range(118)])
element_symbols = {}
for symbol,mass in mda.topology.tables.masses.items():
  
  if len(symbol)==2:
    a,b = symbol[0],symbol[1]
    element_symbols[a.upper()+b.upper()] = symbol
    element_symbols[a.upper()+b.lower()] = symbol
  else:
    element_symbols[symbol] = symbol
    
def guess_bond_order_mda(mol):
    newmol = None
    success = True
    skip = (len(mol.GetBonds())==0)
    if skip:
      success = True
      newmol = mol
    else:
      try:
        m = copy.deepcopy(mol)
        params = Chem.rdmolops.AdjustQueryParameters()
        params.makeBondsGeneric = True
        modmol = Chem.rdmolops.AdjustQueryProperties(m, params)
        modmol.UpdatePropertyCache()
        pdb_string= Chem.MolToPDBBlock(modmol)
        tmp = tempfile.NamedTemporaryFile('w+t')
        tmp.write(pdb_string)
        tmp.seek(0)
        u= mda.Universe(tmp.name,format="PDB")
        tmp.close()
        elements = mda.topology.guessers.guess_types(u.atoms.names)


        # convert symbols to those mda anticipates
        if not set(list(elements)).issubset(set(list(element_symbols.keys()))):
          success = False
        if success:
          elements = np.array([element_symbols[symbol] for symbol in elements],dtype=object)
          u.add_TopologyAttr('elements', elements)
          element_set = set([a.GetSymbol() for a in mol.GetAtoms()])
          force = ("H" not in element_set)
          newmol = u.atoms.convert_to("RDKIT",force=force)
          if newmol is not None:
            success = True
          else:
            success = False
      except:
        success = False

            
#       pieces = Chem.GetMolFrags(mol, asMols=True,sanitizeFrags=False)
#       success_list = []
#       new_mol_list = []
#       for piece in pieces:
#         try:
#           pdb_string= Chem.MolToPDBBlock(piece)
#           tmp = tempfile.NamedTemporaryFile('w+t')
#           tmp.write(pdb_string)
#           tmp.seek(0)
#           u= mda.Universe(tmp.name,format="PDB")
#           tmp.close()
#           elements = mda.topology.guessers.guess_types(u.atoms.names)
#           # convert symbols to those mda anticipates
#           elements = np.array([element_symbols[symbol] for symbol in elements],dtype=object)
#           u.add_TopologyAttr('elements', elements)
#           element_set = set([a.GetSymbol() for a in mol.GetAtoms()])
#           force = ("H" not in element_set)
#           newmol = u.atoms.convert_to("RDKIT",force=force)
#           if newmol is not None:
#             success = True
#           else:
#             success = False
#         except:
#           success = False
#           newmol = None
#         success_list.append(success)
#         new_mol_list.append(newmol)
#     if not all(success_list):
#       success = False
#       newmol = None
#     else:
#       success = True
#       newmol = new_mol_list[0]
#       if len(new_mol_list)>1:
#         for new_mol in new_mol_list[1:]:
#           newmol = Chem.CombineMols(newmol,new_mol)
    return newmol,success

In [34]:
import time
import warnings
fail_reasons = {0:"none",
               1:"Explicit valence error",
               2:"Kekulize error",
               3:"unknown"}

work = mol_dicts
def worker(mol_data):
  Chem.WrapLogs()
  AllChem.WrapLogs()
  with warnings.catch_warnings(record=True) as w:
    warnings.simplefilter("always")
    with redirect_stderr(StringIO()) as err:
      with redirect_stdout(StringIO()) as out:
        filename = mol_data["filename"]
        mol = Chem.MolFromMolFile(filename,removeHs=False,sanitize=False)
        mol_data["mol_guess"] = None
    
        if mol == None:
          mol_data["failed"]=True
        else:
          mol_data["failed"]=False
          mol_data["mol"] = mol
          newmol,success = guess_bond_order_mda(mol)
          if not success:
            mol_data["failed"]=True
          else:
            mol_data["mol_guess"] = newmol
      err = err.getvalue()
      mol_data["err"] = err
      if "Explicit valence" in err:
        failed = True
        mol_data["fail_message"] = fail_reasons[1]
      elif "Can't kekulize" in err:
        failed = True
        mol_data["fail_message"] = fail_reasons[2]
      elif len(err)>0:
        failed = True
        mol_data["fail_message"] = fail_reasons[3]
      else:
        mol_data["fail_message"] = fail_reasons[0]
      mol_data["out"] = out.getvalue()
      mol_data["warnings"] = [str(wmsg.message) for wmsg in w]
      mol_data["timeout"] = False
      if len(mol_data["warnings"])>0:
        for wmsg in mol_data["warnings"]:
          if "reasonable number of iterations" in wmsg:
            mol_data["timeout"] = True
            mol_data["failed"] = True
    return mol_data

In [35]:
with closing(Pool(processes=16)) as pool:
  results = []
  for result in tqdm.tqdm(pool.imap_unordered(worker, work), total=len(work)):
      results.append(result)
  pool.terminate()

100%|██████████| 76684/76684 [02:56<00:00, 434.46it/s]


In [36]:
failed = [mol_dict for mol_dict in results if mol_dict["failed"]]
success = [mol_dict for mol_dict in results if not mol_dict["failed"]]
timeout = [mol_dict for mol_dict in results if mol_dict["timeout"]]
print("Kept:",len(success))
print("Failed:",len(failed))
print("Timeout:",len(timeout))

Kept: 67599
Failed: 9085
Timeout: 5810


In [37]:
failed_messages = [mdata["fail_message"] for mdata in results]
fail_counter = {message:0 for message in fail_reasons.values()}
for message in failed_messages:
  fail_counter[message]+=1
  
print("\nReasons for failure:")
fail_check = 0
for key,value in fail_counter.items():
  print("  ",key.ljust(60," "),":",value)
  fail_check+=value
print("Fail check:",fail_check)


Reasons for failure:
   none                                                         : 73874
   Explicit valence error                                       : 2807
   Kekulize error                                               : 2
   unknown                                                      : 1
Fail check: 76684


In [38]:
final_success = [mdata for mdata in success if mdata["fail_message"]=="none"]

In [39]:
for mdata in final_success:
  with redirect_stderr(StringIO()) as err:
    with redirect_stdout(StringIO()) as out:
      mblock = Chem.MolToMolBlock(mdata["mol_guess"])
      mol = Chem.MolFromMolBlock(mblock,removeHs=False,sanitize=True)
      err = err.getvalue()
      out = out.getvalue()
      if len(err)>0 or len(out)>0 or mol==None:
        print("breaking")
        break


In [40]:
final_dir = Path(processing_dir,"COD_finalized")
if final_dir.exists():
  shutil.rmtree(final_dir)

In [41]:
for mdata in final_success:    
  mdata["incif"] = Path(raw_dir,subtract_paths(Path(mdata["filename"]),filtered_dir)).with_suffix(".cif")
  outmol = Path(final_dir,subtract_paths(Path(mdata["filename"]),filtered_dir))
  outmol.parent.mkdir(exist_ok=True,parents=True)
  mol = mdata["mol_guess"]
  Chem.MolToMolFile(mol,str(outmol))

# STOP!